time to start think the research question \
ex1. What are the main keywords in this journal, and how are they related? \
ex2. How have keyword trends in this journal changed over time? \
-> need to decide how to extract keyword. by word frequency? tf-idf? \
-> can the keyword be clustered? \
   or can we find clustering distribution of the articles then find keywords for each cluster?

In [3]:
###' ################################################################################
###'
###' IMPORT LIBRARIES
###'
###'

### pandas and numpy
import pandas as pd
import numpy as numpy
import spacy

### punctuation, stop words and English language model
from string import punctuation
from spacy.lang.en.stop_words import STOP_WORDS
from spellchecker import SpellChecker
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from PIL import Image
import en_core_web_sm
nlp = en_core_web_sm.load()
import scattertext as st
import re

### textblob
from textblob import TextBlob

### countvectorizer, tfidfvectorizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn import utils
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.preprocessing import LabelBinarizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.manifold import TSNE
from sklearn_extra.cluster import KMedoids
from sklearn.cluster import KMeans

### tqdm
from tqdm import tqdm

### gensim
import gensim
from gensim import models

### PCA
import random
from adjustText import adjust_text

### plotting
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

### kMeans and silhouette scores
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
import matplotlib.cm as cm

### ignore warnings
import warnings
warnings.filterwarnings('ignore')

###time
from datetime import datetime

ModuleNotFoundError: No module named 'sklearn_extra'

In [2]:
import os
os.getcwd()

'C:\\Users\\OWNER\\TopicModeling'

In [3]:
# load the data
# ----------


df_14 = pd.read_csv("articles_data_14.csv")
df_24 = pd.read_csv("articles_data_24.csv")
    
df_14['Published'] = 0  # Code 0 for data from df_14
df_24['Published'] = 1  # Code 1 for data from df_24

df_14 = df_14[df_14['Abstract'] != 'Abstract not available.']
df_24 = df_24[df_24['Abstract'] != 'Abstract not available.']


df_14_S = df_14[['Title','Abstract','Published','Full Text']].dropna()
df_14_S['Abstract'] = df_14_S['Abstract'].apply(lambda x: re.sub(r'<\d+|\*|†>', '', re.sub(r'\<.*?\>', '', x)))

df_24_S = df_24[['Title','Abstract','Published','Full Text']].dropna()
df_24_S['Abstract'] = df_24_S['Abstract'].apply(lambda x: re.sub(r'<\d+|\*|†>', '', re.sub(r'\<.*?\>', '', x)))

#df_S.head()
#len(df_S)

In [ ]:
#df['Abstract'] = df['Abstract'].fillna("")

In [ ]:
# tokenize

###' ################################################################################
###'
###' Function for Deleteing Punctuations and StopWords
###'
###'

### define fuction
def rem_punc_stop(text):
    # When text is None
    if text is None:
        return []

    # Convert text to lowercase
    text = text.lower()

    # Define additional stop words
    stop_words = STOP_WORDS | {"abstract", "available", "student", "research", "study", "impact", "effect",
                               "result", "al", "et", "doi", "googlescholar", "google", "scholar", "textgoogle", 
                               "full", "crossref", "introduction", "background", "purpose" "aim", "objective","use","child"}
    # Define punctuation
    punc = set(punctuation)

    # Remove punctuation
    punc_free = "".join([ch for ch in text if ch not in punc])

    # Apply NLP processing
    doc = nlp(punc_free)

    # Tokenize and lemmatize
    text_lemma = " ".join([token.lemma_ for token in doc])

    # Filter tokens to remove URLs, stop words, and non-alphabetic tokens
    filtered_tokens = [word for word in text_lemma.split() if word not in stop_words and word.isalpha()]

    # Return filtered tokens for TfidfVectorizer
    return filtered_tokens
    

###' ################################################################################
###'
###' Apply the Function and Tokenize Text Column
###'
###'

### sample from the whole dataset
df_24_S['Title_tokens'] = df_24_S['Title'].map(lambda x: rem_punc_stop(x))
df_24_S['Title_join'] = df_24_S['Title_tokens'].map(lambda text: ' '.join(text) if isinstance(text, list) else "")

df_24_S['Abstract_tokens'] = df_24_S['Abstract'].map(lambda x: rem_punc_stop(x))
df_24_S['Abstract_join'] = df_24_S['Abstract_tokens'].map(lambda text: ' '.join(text) if isinstance(text, list) else "")


#df_24_S['Full_tokens'] = df_24_S['Full Text'].map(lambda x: rem_punc_stop(x))
#df_24_S['Full_join'] = df_24_S['Full_tokens'].map(lambda text: ' '.join(text) if isinstance(text, list) else "")


###' ################################################################################
###'
###' Filter Yet Published Papers
###'
###'

#df_S['Full_count'] = df_S['Full Text'].dropna().apply(lambda x: len(str(x).split()))
#df_S = df_S[df_S['Full_count'] >= 2000]

df_S

# 3. TF-IDF

In [ ]:
len(df_S)

In [ ]:
X = df_24_S['Full Text']
tf = TfidfVectorizer(tokenizer = rem_punc_stop,  # specify our function for remove punc and stop words
                     token_pattern = None)       # specify "None" to remove warning. Is this necessary?
tfidf_matrix =  tf.fit_transform(X)

# modify the output to be a dense matrix
dense_matrix = tfidf_matrix.todense()
dense_matrix

In [ ]:
tfidf_df = pd.DataFrame(data = tfidf_matrix.toarray(),       # convert to array than to datafram
                        columns=tf.get_feature_names_out())
# sort by term frequency on the first document
tfidf_df.T.nlargest(10,  # transpose the matrix = columns become documents and rows are words
                     0)  # on column index 0 to show the largest

# 3.1 Clustering

In [ ]:
distortions = []
for i in tqdm(range(2, 11)):
    kmedoids = KMedoids(n_clusters=i, random_state=42).fit(tfidf_matrix)
    #inertia가 군집 내의 분산을 의미
    distortions.append(kmedoids.inertia_)

# plot
plt.figure(figsize=(7,5))
plt.plot(range(2, 11), distortions, marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('Distortion')
plt.title('Elbow Method for Optimal K')
plt.show()

### 4 > 6

In [ ]:
#
# determining optional k: silhouette scores
# ------------------------------
# define a number of functions

# iterate over a k-means fits to have different clusters
# ---------
def run_KMedoids(max_k, data):
    max_k += 1
    kmedoids_results = dict()
    for k in range(2 , max_k):
        kmedoids = KMedoids(n_clusters=k,
                            random_state=42)
        kmedoids_results.update( {k : kmedoids.fit(data)} )
        
    return kmedoids_results


# calculate average silhouettes scores 
# ---------
# plot silhouettes scores
def printAvg(avg_dict):
    for avg in sorted(avg_dict.keys(), reverse=True):
        print("Avg: {}\tK:{}".format(avg.round(4), avg_dict[avg]))

# plot silhouettes scores       
def plotSilhouette(df, n_clusters, kmeans_labels, silhouette_avg): 
    fig, ax1 = plt.subplots(1)
    fig.set_size_inches(8, 6)
    ax1.set_xlim([-0.2, 1])   # play with this to set x-axis limits
    ax1.set_ylim([0, len(df) + (n_clusters + 1) * 10])
    
    ax1.axvline(x=silhouette_avg, color="red", linestyle="--") # The vertical line for average silhouette score of all the values
    ax1.set_yticks([])  # Clear the yaxis labels / ticks
    ax1.set_xticks([-0.2, 0, 0.2, 0.4, 0.6, 0.8, 1])
    plt.title(("Silhouette analysis for K = %d" % n_clusters), fontsize=10, fontweight='bold')
    
    y_lower = 10
    sample_silhouette_values = silhouette_samples(df, kmeans_labels) # Compute the silhouette scores for each sample
    for i in range(n_clusters):
        ith_cluster_silhouette_values = sample_silhouette_values[kmeans_labels == i]
        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        color = cm.nipy_spectral(float(i) / n_clusters)
        ax1.fill_betweenx(numpy.arange(y_lower, y_upper), 0, ith_cluster_silhouette_values, facecolor=color, edgecolor=color, alpha=0.7)

        ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i)) # Label the silhouette plots with their cluster numbers at the middle
        y_lower = y_upper + 10  # Compute the new y_lower for next plot. 10 for the 0 samples
    plt.show()
    
# put it altogether
def silhouette(kmeans_dict, df, plot=True):
    df = df.to_numpy()
    avg_dict = dict()
    for n_clusters, kmeans in kmeans_dict.items():      
        kmeans_labels = kmeans.predict(df)
        silhouette_avg = silhouette_score(df, kmeans_labels) # Average Score for all Samples
        avg_dict.update( {silhouette_avg : n_clusters} )
    
        if(plot): plotSilhouette(df, n_clusters, kmeans_labels, silhouette_avg)

In [ ]:
k = 7 # choose 8 based on the elbow method result from above

# run the k-means algorithm
kmedoids_results = run_KMedoids(k, data = tfidf_df) 

# plot the silhouette analysis
silhouette(kmedoids_results, tfidf_df) 

In [ ]:
# get the top features from each cluster
def get_top_features_cluster(tf_idf_array, prediction, n_feats):
    labels = numpy.unique(prediction)
    dfs = []
    for label in labels:
        id_temp = numpy.where(prediction==label) # indices for each cluster
        x_means = numpy.mean(tf_idf_array[id_temp], axis = 0) # returns average score across cluster
        sorted_means = numpy.argsort(x_means)[::-1][:n_feats] # indices with top 20 scores
        features = tf.get_feature_names_out()
        best_features = [(features[i], x_means[i]) for i in sorted_means]
        df = pd.DataFrame(best_features, columns = ['features', 'score'])
        dfs.append(df)
    return dfs

# plot them on a barplot
def plotWords(dfs, n_feats):
    plt.figure(figsize=(8, 4))
    for i in range(0, len(dfs)):
        plt.title(("Most Common Words in Cluster {}".format(i)), fontsize=10, fontweight='bold')
        sns.barplot(x = 'score' , y = 'features', orient = 'h' , data = dfs[i][:n_feats])
        plt.show()

In [ ]:
best_result = 7
kmedoids = kmedoids_results.get(best_result)

# processing for plot
tfidf_array = tfidf_df.to_numpy()     # convert dataframe to array
prediction = kmedoids.predict(tfidf_df) # predict cluster using tf-idf dataframe


# plot
n_feats = 20
dfs = get_top_features_cluster(tfidf_array, # specify dataset which is an array
                               prediction,  # make specify prediciton
                               n_feats )    # set number of features 
plotWords(dfs, # specify data for plotting  
          13)  # set number of features

In [ ]:
inertia = []

K = range(2, 11) 
for k in K:
    km = KMeans(n_clusters=k,
                random_state=42, # number of clusters
                init='k-means++', # method for initalization 
                n_init=10)     # number of times the k-means algorithm is run with different centroid seeds
    km.fit(tfidf_matrix)     # fit
    inertia.append(km.inertia_) # pipe inertia calculations into list


# plot results
# ---------
plt.plot(K, inertia, marker='o')
plt.xlabel('k')
plt.xticks(range(1, max(K) + 1, 1))
plt.ylabel('Sum_of_squared_distances')
plt.title('Elbow Method For Optimal k')
plt.show()

### 7 or 9

In [ ]:
#
# determining optional k: silhouette scores
# ------------------------------
# define a number of functions

# iterate over a k-means fits to have different clusters
# ---------
def run_KMeans(max_k, data):
    max_k += 1
    kmeans_results = dict()
    for k in range(2 , max_k):
        kmeans = KMeans(n_clusters = k
                               , init = 'k-means++'
                               , n_init = 10
                               , random_state = 42)

        kmeans_results.update( {k : kmeans.fit(data)} )
        
    return kmeans_results


# calculate average silhouettes scores 
# ---------
# plot silhouettes scores
def printAvg(avg_dict):
    for avg in sorted(avg_dict.keys(), reverse=True):
        print("Avg: {}\tK:{}".format(avg.round(4), avg_dict[avg]))

# plot silhouettes scores       
def plotSilhouette(df, n_clusters, kmeans_labels, silhouette_avg): 
    fig, ax1 = plt.subplots(1)
    fig.set_size_inches(8, 6)
    ax1.set_xlim([-0.2, 1])   # play with this to set x-axis limits
    ax1.set_ylim([0, len(df) + (n_clusters + 1) * 10])
    
    ax1.axvline(x=silhouette_avg, color="red", linestyle="--") # The vertical line for average silhouette score of all the values
    ax1.set_yticks([])  # Clear the yaxis labels / ticks
    ax1.set_xticks([-0.2, 0, 0.2, 0.4, 0.6, 0.8, 1])
    plt.title(("Silhouette analysis for K = %d" % n_clusters), fontsize=10, fontweight='bold')
    
    y_lower = 10
    sample_silhouette_values = silhouette_samples(df, kmeans_labels) # Compute the silhouette scores for each sample
    for i in range(n_clusters):
        ith_cluster_silhouette_values = sample_silhouette_values[kmeans_labels == i]
        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        color = cm.nipy_spectral(float(i) / n_clusters)
        ax1.fill_betweenx(numpy.arange(y_lower, y_upper), 0, ith_cluster_silhouette_values, facecolor=color, edgecolor=color, alpha=0.7)

        ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i)) # Label the silhouette plots with their cluster numbers at the middle
        y_lower = y_upper + 10  # Compute the new y_lower for next plot. 10 for the 0 samples
    plt.show()
    
# put it altogether
def silhouette(kmeans_dict, df, plot=True):
    df = df.to_numpy()
    avg_dict = dict()
    for n_clusters, kmeans in kmeans_dict.items():      
        kmeans_labels = kmeans.predict(df)
        silhouette_avg = silhouette_score(df, kmeans_labels) # Average Score for all Samples
        avg_dict.update( {silhouette_avg : n_clusters} )
    
        if(plot): plotSilhouette(df, n_clusters, kmeans_labels, silhouette_avg)

In [ ]:
k = 7 # choose 8 based on the elbow method result from above

# run the k-means algorithm
kmeans_results = run_KMeans(k,                # set k
                            data = tfidf_df)  # identify data


# plot the silhouette analysis
silhouette(kmeans_results,     # take k-means results
           tfidf_df) 

In [ ]:
# get the top features from each cluster
def get_top_features_cluster(tf_idf_array, prediction, n_feats):
    labels = numpy.unique(prediction)
    dfs = []
    for label in labels:
        id_temp = numpy.where(prediction==label) # indices for each cluster
        x_means = numpy.mean(tf_idf_array[id_temp], axis = 0) # returns average score across cluster
        sorted_means = numpy.argsort(x_means)[::-1][:n_feats] # indices with top 20 scores
        features = tf.get_feature_names_out()
        best_features = [(features[i], x_means[i]) for i in sorted_means]
        df = pd.DataFrame(best_features, columns = ['features', 'score'])
        dfs.append(df)
    return dfs

# plot them on a barplot
def plotWords(dfs, n_feats):
    plt.figure(figsize=(8, 4))
    for i in range(0, len(dfs)):
        plt.title(("Most Common Words in Cluster {}".format(i)), fontsize=10, fontweight='bold')
        sns.barplot(x = 'score' , y = 'features', orient = 'h' , data = dfs[i][:n_feats])
        plt.show()

In [ ]:
best_result = 7
kmeans = kmeans_results.get(best_result)

# processing for plot
tfidf_array = tfidf_df.to_numpy()     # convert dataframe to array
prediction = kmeans.predict(tfidf_df) # predict cluster using tf-idf dataframe


# plot
n_feats = 20
dfs = get_top_features_cluster(tfidf_array, # specify dataset which is an array
                               prediction,  # make specify prediciton
                               n_feats )    # set number of features 
plotWords(dfs, # specify data for plotting  
          13)  # set number of features

# 3.2 Latent Dirichlet Allocation (LDA)

In [ ]:
###' ################################################################################
###'
###' Topic Modeling : Latent Dirichlet Allocation (LDA)
###'
###'


### define function
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("\nlda_{}:".format(topic_idx))
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()
    

In [ ]:
#
# Latent Dirichlet Allocation (LDA)
# ---------------------------------------

# pre-processing
# --------
# create a new data object called X
X = df_S['Title_join']
tf = TfidfVectorizer(tokenizer = rem_punc_stop,  # specify our function for remove punc and stop words
                     token_pattern = None)       # specify "None" to remove warning. Is this necessary?

# apply tf-idf vectorizer to our data (X)
tfidf_matrix =  tf.fit_transform(X)

# modify the output to be a dense matrix
dense_matrix = tfidf_matrix.todense()

# intitialize LDA model and 
# --------
# initialize LDA and set model parameters
lda = LatentDirichletAllocation(n_components=7, # specify the number of components
                                max_iter=20,    # specify the number of iterations 
                                random_state=42) # set a seed for reproducibility

# fit LDA model to our dense matrix
lda = lda.fit(numpy.asarray(dense_matrix))

# post-processing
# --------
# get feature names from our tf-idf vector
tf_feature_names = tf.get_feature_names_out()

# print top words 
print_top_words(lda,               # specify model
                tf_feature_names,  # specify feature names vector
                20)                # specify how many words we want to see
 

# now transform our data using the lda model and create a dataframe
col_names = ['lda_0', 'lda_1','lda_2', 'lda_3','lda_4','lda_5', 'lda_6']
topic_dist = lda.transform(tfidf_matrix)
topic_dist_df = pd.DataFrame(topic_dist, columns = col_names).reset_index(drop = True)

# view the corresponding tf-idf dataframe with tf-idf values
topic_dist_df

In [ ]:
df_w_topics = df_S.join(topic_dist_df.reset_index())
df_w_topics.head(5)

# 3.3 Non-Negative Matrix Factorization (NMF)

In [ ]:
###' ################################################################################
###'
###' Topic Modeling : on-Negative Matrix Factorization (NMF)
###'
###'


### define function
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("\nnmf_{}:".format(topic_idx))
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()
    
    
    
# pre-processing
Y = df_S['Abstract_join']

tf = TfidfVectorizer(tokenizer = rem_punc_stop,
                     token_pattern = None) 

tfidf_mx =  tf.fit_transform(Y)
dense_matrix = tfidf_mx.todense()


# initialize LDA and set model parameters
nmf = NMF(n_components=7,  # specify the number of components
          init='random',   # specify the initalization method
          random_state=42)  # set a seed for reproducibility

# fit NMF model to our dense matrix
nmf = nmf.fit(numpy.asarray(dense_matrix))

# post-processing
tf_feature_names = tf.get_feature_names_out()

In [ ]:
### print top words 
print_top_words(nmf, tf_feature_names,20)  

### join
col_names2 = ['nmf_0', 'nmf_1', 'nmf_2', 'nmf_3', 'nmf_4', 'nmf_5', 'nmf_6']
topic_dist2 = nmf.transform(tfidf_mx)
topic_dist_nmf = pd.DataFrame(topic_dist2, columns = col_names2).reset_index(drop = True)
topic_dist_nmf

#df_w_topics2 = topic_dist_df.join(topic_dist_nmf.reset_index())
#df_w_topics2.head(5)

In [ ]:
# PCA

In [ ]:
df_24_S

In [ ]:
pca = PCA(n_components=2)

X_train = pca.fit_transform(df_24_S['Abstract'])

explained_variance = pca.explained_variance_ratio_